# IsoNet
------
This will be used to create the model for predicting precipitable stable water isotopes. Only creates and exports the model and trains it. Is not used to predict.

Table of Contents:
1. [Importing Data](#importing-data)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Creation](#model-creation)
4. [Model Training](#model-training)
5. [Model Export](#model-export)

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime
import matplotlib.pyplot as plt

# Tensorflow and keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

2024-01-07 19:25:55.933024: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 19:25:56.683276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 19:25:56.683339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 19:25:57.115816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 19:25:57.701168: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 19:25:57.705754: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Importing Data
Importing the data, and then splitting it into features and labels

In [2]:
# Load in the data, separate the features and labels
data = pd.read_csv(r'Isoscape_Data.csv')
features = data.drop(['H2avg', 'dex', 'O18Avg', 'Station'], axis=1)
labels = data['O18Avg']

# Adjusting the date time to be int values where Janurary 1st, of year 1 is 1
features['Date'] = pd.to_datetime(features['Date'])
features['Date'] = features['Date'].map(datetime.datetime.toordinal)

numFeatures = features.shape[1]

## Data Preprocessing
Preprocessing the data, including:
* Convert into numpy arrays
* Split into training and testing sets
* Convert into tensorflow Datasets
* Windowing the data (Skipping for now, will do later) 
* Standardizing the data

In [3]:
# Split the data into training and testing sets
splitIndex = int(0.8 * len(features))
xTrain = features[:splitIndex]
xTest = features[splitIndex:]
yTrain = labels[:splitIndex]
yTest = labels[splitIndex:]

# Scale the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Convert the data into numpy arrays
xTrain = np.array(xTrain)
xTest = np.array(xTest)
yTrain = np.array(yTrain)
yTest = np.array(yTest)

# Convert traindata into tensor dataset
trainData = tf.data.Dataset.from_tensor_slices((xTrain, yTrain))

## Model Creation
Creating the model, including:
* Creating the model architecture
* Compiling the model

Model architecture:
* Input layer
* 1 LSTM layer
* 2 Dense layers
* Output layer

In [4]:
numNeuorns = 64
model = Sequential()
model.add(InputLayer(input_shape=(numFeatures, 1)))
model.add(LSTM(numNeuorns))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

## Model Training
Training and evaluating the model

In [5]:
es = EarlyStopping(monitor='mean_absolute_error', patience=10)

batchSize = 64
model.fit(
    trainData.batch(batchSize),
    batch_size=batchSize,
    epochs=1000,
    callbacks=[es]
)

Epoch 1/1000


23/23 [==============================] - 3s 6ms/step - loss: 384.2643 - root_mean_squared_error: 19.6027 - mean_absolute_error: 17.6712
Epoch 2/1000
23/23 [==============================] - 0s 6ms/step - loss: 330.4948 - root_mean_squared_error: 18.1795 - mean_absolute_error: 15.8414
Epoch 3/1000
23/23 [==============================] - 0s 7ms/step - loss: 112.0668 - root_mean_squared_error: 10.5862 - mean_absolute_error: 8.3929
Epoch 4/1000
23/23 [==============================] - 0s 6ms/step - loss: 51.0048 - root_mean_squared_error: 7.1418 - mean_absolute_error: 5.5718
Epoch 5/1000
23/23 [==============================] - 0s 7ms/step - loss: 38.1879 - root_mean_squared_error: 6.1796 - mean_absolute_error: 4.9281
Epoch 6/1000
23/23 [==============================] - 0s 6ms/step - loss: 32.8818 - root_mean_squared_error: 5.7343 - mean_absolute_error: 4.5963
Epoch 7/1000
23/23 [==============================] - 0s 6ms/step - loss: 31.3559 - root_mean_squared_error: 5.5996 - mean_absolu

## Model Export

In [6]:
# Printing how well the model did on the test data
print(model.evaluate(xTest, yTest))

12/12 [==============================] - 1s 4ms/step - loss: 16.1657 - root_mean_squared_error: 4.0207 - mean_absolute_error: 3.1553
[16.165679931640625, 4.020656585693359, 3.1552772521972656]


In [7]:
# Create new pd DataFrame with the predictions and actual values including the features
predictions = model.predict(xTest)
df = features.loc[splitIndex:]
df['Predictions'] = predictions
df['Actual'] = yTest

# Export the data to a csv file
df.to_csv('results_test.csv', index=False)

12/12 [==============================] - 0s 3ms/step


/tmp/ipykernel_640/3509207634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predictions'] = predictions
/tmp/ipykernel_640/3509207634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual'] = yTest


In [17]:
# Save the model
model.save('printModel.keras')

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                16896     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25281 (98.75 KB)
Trainable params: 25281 (98.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
